In [5]:
#import libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
df=pd.read_csv('train.csv')
nf=pd.read_csv('validation.csv')

In [7]:
df.columns = df.columns.str.strip()
nf.columns = nf.columns.str.strip()

In [8]:
df.shape

(5000, 62)

In [9]:
df.drop(['id'],axis=1,inplace=True)
df.drop(['url'],axis=1,inplace=True)

In [10]:
nf.drop(['id'],axis=1,inplace=True)
nf.drop(['url'],axis=1,inplace=True)

In [11]:
df_Train=df.iloc[0:,:]
df_Test=nf.iloc[0:,:]

In [12]:
df_Test.drop(['shares'],axis=1,inplace=True)

C:\Users\dfgbe\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [13]:
df_Test.shape

(1000, 59)

In [14]:
x_train=df_Train.drop(['shares'],axis=1)
y_train=df_Train['shares']

In [24]:
import xgboost 
classifier=xgboost.XGBRegressor()
#classifier.fit(x_train,y_train)

In [25]:
import xgboost
regressor=xgboost.XGBRegressor()

In [26]:
booster=['gbtree','gblinear']
base_score=[0.25,0.5,0.75,1]

In [27]:
n_estimators = [100, 500, 900, 1100, 1500]
max_depth = [2, 3, 5, 10, 15]
booster=['gbtree','gblinear']
learning_rate=[0.05,0.1,0.15,0.20]
min_child_weight=[1,2,3,4]

hyperparameter_grid = {
    'n_estimators': n_estimators,
    'max_depth':max_depth,
    'learning_rate':learning_rate,
    'min_child_weight':min_child_weight,
    'booster':booster,
    'base_score':base_score
}

In [30]:
from sklearn.model_selection import RandomizedSearchCV

random_cv = RandomizedSearchCV(estimator=regressor,
            param_distributions=hyperparameter_grid,
            cv=5, n_iter=50,
            scoring = 'neg_mean_absolute_error', n_jobs = 4,
            verbose = 5,
            return_train_score = True,
            random_state = 42)


In [31]:
random_cv.fit(x_train,y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:   33.2s
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:  7.4min
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed: 12.0min
[Parallel(n_jobs=4)]: Done 250 out of 250 | elapsed: 14.8min finished


[10:02:09] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                          colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=1, gamma=0,
                                          importance_type='gain',
                                          learning_rate=0.1, max_delta_step=0,
                                          max_depth=3, min_child_weight=1,
                                          missing=None, n_estimators=100,
                                          n_jobs=1, nthread=None,
                                          objective='reg:linear',
                                          random_state=0, reg_alpha=...
                   iid='deprecated', n_iter=50, n_jobs=4,
                   param_distributions={'base_score': [0.25, 0.5, 0.75, 1],
                                        'booster': ['g

In [32]:
random_cv.best_estimator_

XGBRegressor(base_score=0.25, booster='gblinear', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=4, missing=None, n_estimators=1100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [33]:
regressor=xgboost.XGBRegressor(base_score=0.25, booster='gblinear', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=4, missing=None, n_estimators=1100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [34]:
regressor.fit(x_train,y_train)

[10:08:26] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.25, booster='gblinear', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=4, missing=None, n_estimators=1100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [35]:
import pickle
filename = 'finalized_model.pkl'
pickle.dump(classifier,open(filename,'wb'))

In [37]:
y_pred=regressor.predict(df_Test)

In [40]:
y_pred

array([ 2008.1556 ,  3805.1055 ,  1593.3964 ,  3119.2427 ,  2903.9421 ,
        2281.2107 ,  3961.6245 ,  3131.9353 ,  2760.266  ,  1443.113  ,
        2765.7485 ,  6143.937  ,  2301.7837 ,  2922.5728 ,  5293.079  ,
        2862.6003 ,  2266.796  ,  3764.5337 ,  2416.1182 ,  2941.2861 ,
        1835.5144 ,  3313.9636 ,  3939.4856 ,  3006.2668 ,  2168.3198 ,
        2444.818  ,  2020.1921 ,  2397.2134 ,  3107.4634 ,  3180.3582 ,
        2881.2852 ,  4017.6208 ,  3234.4412 ,  2133.9045 ,  4343.807  ,
        2857.626  ,  3343.316  ,  5144.4077 ,  5260.9736 ,  3328.548  ,
        3222.242  ,  2238.3276 ,  2295.8176 ,  2587.6243 ,  5300.913  ,
        3152.892  ,  2687.4634 ,  4703.2007 ,  1794.3013 ,  4120.181  ,
        5080.7583 ,  5554.434  ,  3515.063  ,  3248.7832 ,  2924.6433 ,
        7514.6255 ,  2694.819  ,  2665.3042 ,  3603.2576 ,  2230.12   ,
        6986.5425 ,  3733.2317 ,  1178.0173 ,  4788.1426 ,  6826.5605 ,
        2384.8857 ,  4253.1157 ,  3305.4224 ,  2275.6082 ,  1465

In [42]:
corr=df_Train.corr()

In [43]:
corr

,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
timedelta,1.000000,-0.243861,-0.062606,-0.011415,-0.012354,-0.011096,0.001155,0.084135,-0.001696,-0.002302,...,0.078065,0.107565,-0.027320,0.055668,-0.085431,-0.011182,0.053576,0.015111,0.010796,-0.011844
n_tokens_title,-0.243861,1.000000,0.016624,-0.010082,-0.009861,-0.010093,-0.044708,-0.005632,-0.020840,0.070137,...,-0.042524,-0.020748,-0.002784,-0.026924,0.024747,0.083849,0.003470,-0.150208,0.054882,0.010694
n_tokens_content,-0.062606,0.016624,1.000000,0.025534,0.033500,0.027342,0.453081,0.319458,0.365091,0.085006,...,-0.259488,0.410739,-0.127551,-0.432253,0.230331,0.011605,0.008783,0.006558,0.003933,-0.025070
n_unique_tokens,-0.011415,-0.010082,0.025534,1.000000,0.999946,0.999981,-0.001368,0.024778,0.076078,-0.003933,...,-0.012719,-0.040915,0.025734,0.025433,0.011709,-0.012074,-0.003994,-0.026061,-0.009672,0.004472
n_non_stop_words,-0.012354,-0.009861,0.033500,0.999946,1.000000,0.999942,0.002370,0.027346,0.079556,-0.003970,...,-0.015892,-0.036938,0.024621,0.021593,0.013944,-0.012317,-0.003886,-0.025810,-0.009818,0.004003
n_non_stop_unique_tokens,-0.011096,-0.010093,0.027342,0.999981,0.999942,1.000000,-0.001454,0.025153,0.074568,-0.004042,...,-0.012904,-0.038530,0.024402,0.023428,0.011911,-0.012543,-0.004321,-0.025816,-0.010055,0.004024
num_hrefs,0.001155,-0.044708,0.453081,-0.001368,0.002370,-0.001454,1.000000,0.374062,0.373871,0.096269,...,-0.094226,0.301379,-0.154175,-0.264165,0.050412,0.047865,0.037257,-0.001619,0.058085,0.032497
num_self_hrefs,0.084135,-0.005632,0.319458,0.024778,0.027346,0.025153,0.374062,1.000000,0.281426,0.027783,...,-0.062645,0.204620,-0.052529,-0.100594,0.032277,-0.002725,0.026615,-0.005920,0.004348,-0.009001
num_imgs,-0.001696,-0.020840,0.365091,0.076078,0.079556,0.074568,0.373871,0.281426,1.000000,-0.071728,...,-0.037248,0.175201,-0.074474,-0.135062,0.037679,0.062114,0.047269,-0.017980,0.069438,0.063019
num_videos,-0.002302,0.070137,0.085006,-0.003933,-0.003970,-0.004042,0.096269,0.027783,-0.071728,1.000000,...,-0.012927,0.116689,-0.117901,-0.140754,0.026423,0.081046,0.029413,-0.050267,0.058370,0.015876


In [46]:
columns = np.full((corr.shape[0],), True, dtype=bool)
for i in range(corr.shape[0]):
    for j in range (i+1, corr.shape[0]):
        if corr.iloc[i,j] >= 0.9:
            if columns[j]:
                columns[j] = False


In [47]:
selected_columns = df_Train.columns[columns]
data = df_Train[selected_columns]

In [48]:
columns

array([ True,  True,  True,  True, False, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

In [67]:
import csv
share_count = 0

with open('sample_submission2.csv', 'w', newline='') as csvfile:
    
    fieldnames = ['id','shares']
    thewriter = csv.DictWriter(csvfile, fieldnames=fieldnames)
    thewriter.writeheader()
    
    for shares in ann_pred:
        share_count += 1
        thewriter.writerow({'id':share_count, 'shares':shares})

In [63]:
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU,PReLU,ELU
from keras.layers import Dropout


# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(output_dim = 50, init = 'he_uniform',activation='relu',input_dim = 59))

# Adding the second hidden layer
classifier.add(Dense(output_dim = 25, init = 'he_uniform',activation='relu'))

# Adding the third hidden layer
classifier.add(Dense(output_dim = 50, init = 'he_uniform',activation='relu'))
# Adding the output layer
classifier.add(Dense(output_dim = 1, init = 'he_uniform'))

# Compiling the ANN
classifier.compile(loss=root_mean_squared_error, optimizer='Adamax')

# Fitting the ANN to the Training set
model_history=classifier.fit(x_train.values, y_train.values,validation_split=0.20, batch_size = 10, nb_epoch = 1000)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


C:\Users\dfgbe\anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=59, units=50, kernel_initializer="he_uniform")`
  del sys.path[0]
C:\Users\dfgbe\anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=25, kernel_initializer="he_uniform")`
  app.launch_new_instance()
C:\Users\dfgbe\anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=50, kernel_initializer="he_uniform")`
C:\Users\dfgbe\anaconda3\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1, kernel_initializer="he_uniform")`
C:\Users\dfgbe\anaconda3\lib\site-packages\ipykernel_launcher.py:27: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.



Train on 4000 samples, validate on 1000 samples
Epoch 1/1000
4000/4000 [==============================] - 0s 118us/step - loss: 8833.5787 - val_loss: 6335.1604
Epoch 2/1000
4000/4000 [==============================] - 0s 79us/step - loss: 6037.8473 - val_loss: 6701.5905
Epoch 3/1000
4000/4000 [==============================] - 0s 83us/step - loss: 5714.9799 - val_loss: 5705.2977
Epoch 4/1000
4000/4000 [==============================] - 0s 86us/step - loss: 5260.5407 - val_loss: 5730.1547
Epoch 5/1000
4000/4000 [==============================] - 0s 88us/step - loss: 5296.7786 - val_loss: 5687.8912
Epoch 6/1000
4000/4000 [==============================] - 0s 90us/step - loss: 5272.3281 - val_loss: 5483.3916
Epoch 7/1000
4000/4000 [==============================] - 0s 86us/step - loss: 5234.2627 - val_loss: 5466.1653
Epoch 8/1000
4000/4000 [==============================] - 0s 81us/step - loss: 5088.8831 - val_loss: 5688.1686
Epoch 9/1000
4000/4000 [==============================] - 0s 8

Epoch 72/1000
4000/4000 [==============================] - 0s 96us/step - loss: 4657.3325 - val_loss: 5379.1053
Epoch 73/1000
4000/4000 [==============================] - 0s 90us/step - loss: 4562.3388 - val_loss: 5326.5016
Epoch 74/1000
4000/4000 [==============================] - 0s 79us/step - loss: 4603.7167 - val_loss: 5335.8071
Epoch 75/1000
4000/4000 [==============================] - 0s 84us/step - loss: 4590.7499 - val_loss: 5356.9001
Epoch 76/1000
4000/4000 [==============================] - 0s 80us/step - loss: 4675.3365 - val_loss: 5356.9060
Epoch 77/1000
4000/4000 [==============================] - 0s 80us/step - loss: 4658.2879 - val_loss: 5362.0756
Epoch 78/1000
4000/4000 [==============================] - 0s 82us/step - loss: 4552.3671 - val_loss: 5331.8274
Epoch 79/1000
4000/4000 [==============================] - 0s 81us/step - loss: 4619.8739 - val_loss: 5340.7592
Epoch 80/1000
4000/4000 [==============================] - 0s 80us/step - loss: 4593.3917 - val_loss: 53

4000/4000 [==============================] - 0s 79us/step - loss: 4435.8000 - val_loss: 5325.6898
Epoch 145/1000
4000/4000 [==============================] - 0s 88us/step - loss: 4529.4684 - val_loss: 5392.8454
Epoch 146/1000
4000/4000 [==============================] - 0s 75us/step - loss: 4624.5642 - val_loss: 5317.4378
Epoch 147/1000
4000/4000 [==============================] - 0s 80us/step - loss: 4546.1156 - val_loss: 5324.4434
Epoch 148/1000
4000/4000 [==============================] - 0s 81us/step - loss: 4589.7148 - val_loss: 5334.8021
Epoch 149/1000
4000/4000 [==============================] - 0s 95us/step - loss: 4541.2822 - val_loss: 5316.0222
Epoch 150/1000
4000/4000 [==============================] - 0s 94us/step - loss: 4511.5452 - val_loss: 5337.6754
Epoch 151/1000
4000/4000 [==============================] - 0s 77us/step - loss: 4569.0329 - val_loss: 5352.0261
Epoch 152/1000
4000/4000 [==============================] - 0s 85us/step - loss: 4503.4352 - val_loss: 5320.569

...

4000/4000 [==============================] - 0s 117us/step - loss: 4435.9402 - val_loss: 5425.9079
Epoch 649/1000
4000/4000 [==============================] - 0s 87us/step - loss: 4421.4929 - val_loss: 5401.5181
Epoch 650/1000
4000/4000 [==============================] - 0s 98us/step - loss: 4446.3529 - val_loss: 5407.4146
Epoch 651/1000
4000/4000 [==============================] - 0s 104us/step - loss: 4415.6810 - val_loss: 5403.9889
Epoch 652/1000
4000/4000 [==============================] - 0s 109us/step - loss: 4423.2263 - val_loss: 5378.7601
Epoch 653/1000
4000/4000 [==============================] - 0s 105us/step - loss: 4421.9801 - val_loss: 5443.3149
Epoch 654/1000
4000/4000 [==============================] - 0s 88us/step - loss: 4476.6999 - val_loss: 5359.9456
Epoch 655/1000
4000/4000 [==============================] - 0s 107us/step - loss: 4493.6966 - val_loss: 5356.7496
Epoch 656/1000
4000/4000 [==============================] - 0s 109us/step - loss: 4431.2597 - val_loss: 54

4000/4000 [==============================] - 0s 110us/step - loss: 4391.4126 - val_loss: 5415.2778
Epoch 721/1000
4000/4000 [==============================] - 0s 91us/step - loss: 4421.1484 - val_loss: 5395.4688
Epoch 722/1000
4000/4000 [==============================] - 0s 94us/step - loss: 4466.4957 - val_loss: 5410.6093
Epoch 723/1000
4000/4000 [==============================] - 0s 88us/step - loss: 4447.0259 - val_loss: 5386.8160
Epoch 724/1000
4000/4000 [==============================] - 0s 85us/step - loss: 4459.1019 - val_loss: 5384.9667
Epoch 725/1000
4000/4000 [==============================] - 0s 88us/step - loss: 4408.2677 - val_loss: 5395.1689
Epoch 726/1000
4000/4000 [==============================] - 0s 96us/step - loss: 4472.3421 - val_loss: 5381.3178
Epoch 727/1000
4000/4000 [==============================] - 0s 89us/step - loss: 4404.1388 - val_loss: 5384.4628
Epoch 728/1000
4000/4000 [==============================] - 0s 88us/step - loss: 4415.7584 - val_loss: 5417.45

4000/4000 [==============================] - 0s 91us/step - loss: 4416.2644 - val_loss: 5373.4082
Epoch 793/1000
4000/4000 [==============================] - 0s 85us/step - loss: 4271.6531 - val_loss: 5381.4559
Epoch 794/1000
4000/4000 [==============================] - 0s 90us/step - loss: 4507.9280 - val_loss: 5409.4658
Epoch 795/1000
4000/4000 [==============================] - 0s 95us/step - loss: 4399.8217 - val_loss: 5365.6190
Epoch 796/1000
4000/4000 [==============================] - 0s 98us/step - loss: 4367.4297 - val_loss: 5392.4682
Epoch 797/1000
4000/4000 [==============================] - 0s 118us/step - loss: 4462.1922 - val_loss: 5344.4507
Epoch 798/1000
4000/4000 [==============================] - 1s 131us/step - loss: 4377.8482 - val_loss: 5388.2820
Epoch 799/1000
4000/4000 [==============================] - 0s 112us/step - loss: 4471.9285 - val_loss: 5378.4249
Epoch 800/1000
4000/4000 [==============================] - 0s 106us/step - loss: 4402.0114 - val_loss: 5388

4000/4000 [==============================] - 0s 85us/step - loss: 4407.8853 - val_loss: 5419.6621
Epoch 865/1000
4000/4000 [==============================] - 0s 82us/step - loss: 4431.3834 - val_loss: 5423.5169
Epoch 866/1000
4000/4000 [==============================] - 0s 83us/step - loss: 4462.3807 - val_loss: 5472.4252
Epoch 867/1000
4000/4000 [==============================] - 0s 102us/step - loss: 4479.2413 - val_loss: 5379.7358
Epoch 868/1000
4000/4000 [==============================] - 0s 106us/step - loss: 4390.4747 - val_loss: 5429.2641
Epoch 869/1000
4000/4000 [==============================] - 1s 136us/step - loss: 4396.4971 - val_loss: 5408.7191
Epoch 870/1000
4000/4000 [==============================] - 0s 97us/step - loss: 4443.0540 - val_loss: 5399.3017
Epoch 871/1000
4000/4000 [==============================] - 0s 88us/step - loss: 4440.6188 - val_loss: 5465.5981
Epoch 872/1000
4000/4000 [==============================] - 0s 86us/step - loss: 4443.8577 - val_loss: 5448.

4000/4000 [==============================] - 0s 91us/step - loss: 4429.1229 - val_loss: 5433.8244
Epoch 937/1000
4000/4000 [==============================] - 0s 91us/step - loss: 4346.0364 - val_loss: 5461.2265
Epoch 938/1000
4000/4000 [==============================] - 0s 98us/step - loss: 4409.7000 - val_loss: 5432.3050
Epoch 939/1000
4000/4000 [==============================] - 0s 84us/step - loss: 4358.6786 - val_loss: 5435.8490
Epoch 940/1000
4000/4000 [==============================] - 0s 86us/step - loss: 4402.0036 - val_loss: 5451.0161
Epoch 941/1000
4000/4000 [==============================] - 0s 91us/step - loss: 4455.3791 - val_loss: 5443.1600
Epoch 942/1000
4000/4000 [==============================] - 0s 87us/step - loss: 4334.0008 - val_loss: 5446.6311
Epoch 943/1000
4000/4000 [==============================] - 0s 92us/step - loss: 4415.1642 - val_loss: 5429.7771
Epoch 944/1000
4000/4000 [==============================] - 0s 94us/step - loss: 4378.8605 - val_loss: 5425.953

In [64]:
ann_pred=classifier.predict(df_Test.values)

In [65]:
from keras import backend as K
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [66]:
ann_pred

array([[ 1693.309   ],
       [ 2640.018   ],
       [ 1695.7252  ],
       [ 2512.5837  ],
       [ 2290.5137  ],
       [ 2004.4769  ],
       [ 3026.5894  ],
       [ 1696.5527  ],
       [ 2275.851   ],
       [ 2453.2393  ],
       [ 2363.108   ],
       [ 5386.6094  ],
       [ 2161.4475  ],
       [ 1918.1283  ],
       [ 2532.7222  ],
       [ 1879.8496  ],
       [ 1824.4535  ],
       [ 2477.87    ],
       [ 1898.97    ],
       [ 2183.6372  ],
       [ 1485.6763  ],
       [ 1235.7169  ],
       [ 2193.4023  ],
       [ 2692.8503  ],
       [ 1762.496   ],
       [ 1750.7637  ],
       [ 1584.4362  ],
       [ 1861.0878  ],
       [ 2542.7632  ],
       [ 3993.4429  ],
       [ 2245.629   ],
       [ 3182.5244  ],
       [ 2533.8018  ],
       [ 3157.811   ],
       [ 2953.8145  ],
       [ 2185.122   ],
       [ 2840.4622  ],
       [ 3225.7432  ],
       [ 3685.0671  ],
       [ 2570.9485  ],
       [ 1884.0715  ],
       [ 2007.2186  ],
       [ 1991.7101  ],
       [ 18